In [ ]:
#!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.9 MB/s eta 0:00:00


 Using the RSA_lib.py and AES library, please design the hybrid encryption following the protocol:

  s1. Bob generates the public key/secret key using RSA.

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Random import get_random_bytes
import base64

def aes_encrypt(message, session_key):
    cipher_aes = AES.new(session_key, AES.MODE_GCM)
    nonce = cipher_aes.nonce
    ciphertext, tag = cipher_aes.encrypt_and_digest(message.encode())
    return nonce, ciphertext, tag

def aes_decrypt(nonce, ciphertext, tag, session_key):
    cipher_aes = AES.new(session_key, AES.MODE_GCM, nonce=nonce)
    decrypted_message = cipher_aes.decrypt_and_verify(ciphertext, tag)
    return decrypted_message.decode()


In [ ]:
keyPair = RSA.generate(bits=1024)
public_key = keyPair.publickey()
print(f"Public key: (n={hex(keyPair.n)}, e={hex(keyPair.e)}))")
print(f"Private key: (n={hex(keyPair.n)}, d={hex(keyPair.d)}))")

Public key: (n=0xb52175073bd10e444c9331aa835284a7ade5b26123a2d03a63d943ec84e41c567f90abf67e3b54984a667d4901f1577edc18826c32438f78ce722b66dbef9d200ce1ffa1a3f827178f6be9bfe69bd67d13cad079fac90253223f59dbab49c080d6f8485a3ac7c8f88eef8c448deca7b9a83813450b32ab1eb0df17bf03298e2b, e=0x10001))
Private key: (n=0xb52175073bd10e444c9331aa835284a7ade5b26123a2d03a63d943ec84e41c567f90abf67e3b54984a667d4901f1577edc18826c32438f78ce722b66dbef9d200ce1ffa1a3f827178f6be9bfe69bd67d13cad079fac90253223f59dbab49c080d6f8485a3ac7c8f88eef8c448deca7b9a83813450b32ab1eb0df17bf03298e2b, d=0x4ccdd05f2e30b493fc19b8c84cc1ae18f681478f83c4d8a169bab4353e6f061fc77258f8945e24137a0e2b61cc691713667b1e1288330da07429afd855ba2cd80ae74cd3400f5c8186792162fb173c4f29200262afcf31d6cad4fe7a87b6acff7bba060db990a9dff625219c949e300bf1a15ef35a207ecf0efac0ea50e307d))


s2.     

a.Alice generates the session key and uses Bob's public key to encrypt the session key, then she sends the ciphertext to Bob.
      
b. Alice also uses the session key to encrypt the first message: "Hello, Bob" using the AES mechanism.

In [ ]:
session_key = get_random_bytes(16)
cipher_rsa = PKCS1_OAEP.new(public_key)
encrypted_session_key = cipher_rsa.encrypt(session_key)
print("Encrypted session key:", base64.b64encode(encrypted_session_key).decode())

nonce, ciphertext, tag = aes_encrypt("Hello, Bob", session_key)
print("AES Encrypted message:", base64.b64encode(ciphertext).decode())

Encrypted session key: VhqbYMsWMtM7rfRpi3iGoEoI4JHVEdUSKjDv7w7InmlAPh9sNSGESJRxITo/Bj0wK7VPZX9qAC9pRiEPI7PlLdM9RX/1f1z2Oq8R6tW3/WM5G9YdLLp5rQV1KzIHhS0puhjjsjZm4TT4Ei4vVB/TQCYLu7F8DEoIMyjhzAS4QhI=
AES Encrypted message: aPZ6kKFK6ITe6A==


        
s3.

a. Bob uses the secret key to decrypt the ciphertext and obtain the session key.

b. Bob uses the decrypted session key to decrypt the first message and receives "Hello, Bob".




In [ ]:
cipher_rsa = PKCS1_OAEP.new(keyPair)
decrypted_session_key = cipher_rsa.decrypt(encrypted_session_key)
print("Decrypted session key matches:", decrypted_session_key == session_key)

decrypted_message = aes_decrypt(nonce, ciphertext, tag, decrypted_session_key)
print("Decrypted message:", decrypted_message)

Decrypted session key matches: True
Decrypted message: Hello, Bob


s4. Bob and Alice will communicate together using the session key.

In [ ]:
for i in range(5):
    message = f"Message {i+1} from Bob"
    nonce, ciphertext, tag = aes_encrypt(message, session_key)
    print(f"Round {i+1}: Bob sends encrypted message:", base64.b64encode(ciphertext).decode())

    decrypted_message = aes_decrypt(nonce, ciphertext, tag, session_key)
    print(f"Round {i+1}: Alice decrypts the message and receives:", decrypted_message)

Round 1: Bob sends encrypted message: cvuv6d/bEfVIXbPqD9eY5zfB
Round 1: Alice decrypts the message and receives: Message 1 from Bob
Round 2: Bob sends encrypted message: BsWXU7Pbr4LAnQC+7FmxYH2p
Round 2: Alice decrypts the message and receives: Message 2 from Bob
Round 3: Bob sends encrypted message: SFF1hQYu8CPEqnsoAZHYsVQu
Round 3: Alice decrypts the message and receives: Message 3 from Bob
Round 4: Bob sends encrypted message: FVKscbBor04nbJwGPgaFY78Z
Round 4: Alice decrypts the message and receives: Message 4 from Bob
Round 5: Bob sends encrypted message: sSnv0zR/35agWwNSXQWTwE/O
Round 5: Alice decrypts the message and receives: Message 5 from Bob


s5. After five rounds, Alice will generate the new session key and uses the same protocol to send Bob the new session key.

In [ ]:
new_session_key = get_random_bytes(16)
encrypted_new_session_key = cipher_rsa.encrypt(new_session_key)
print("New session key encrypted and sent to Bob:", base64.b64encode(encrypted_new_session_key).decode())

decrypted_new_session_key = cipher_rsa.decrypt(encrypted_new_session_key)
print("Bob decrypts the new session key. Matches new session key:", decrypted_new_session_key == new_session_key)

New session key encrypted and sent to Bob: aeZteoJtKVS8IThKuxjXUvTMtWYhFifrCYlgkpeeZeO54hsfQLnbUAc4ZD1xQbmHbI6td4hwwlNj6z3JFF02wtbVg7LfGYUOIwniSUMd5ySSm84ISa29P3aYATl2eB2bbnrIclyVxDCo3+AMcIlLIURO7ZV18ARNUTwJxBDF30o=
Bob decrypts the new session key. Matches new session key: True


In [ ]:
message = f"Hey Alice, this is Bob with a new key!"
nonce, ciphertext, tag = aes_encrypt(message, new_session_key)
print(f"Bob sends encrypted message:", base64.b64encode(ciphertext).decode())

decrypted_message = aes_decrypt(nonce, ciphertext, tag, new_session_key)
print(f"Alice decrypts the message and receives:", decrypted_message)

Bob sends encrypted message: Z7sTwIZo7WHGSQ00g92+fnNEKAMUNABFPI59fAz/O2CP3Ck9K70=
Alice decrypts the message and receives: Hey Alice, this is Bob with a new key!
